# Microtones #

This notebook discusses musical scales based on subdividing the octave into something other than the traditional Western 12 equal parts. To run it, you will need my [ipy_magics](https://gitlab.com/ldo/ipy_magics), specifically `csound_magic.py` and `setvar_magic.py`. This uses [Csound](https://csound.com/) to generate the audio samples.

In [ ]:
# Edit this as appropriate for the correct paths to the Python modules
%run ../ipy_magics/setvar_magic.py
%run ../ipy_magics/csound_magic.py

Define some common preamble to simplify writing the Csound scores:

In [ ]:
%%setvar common_setup

    <CsInstruments>

        sr = 44100
        ksmps = 32
        nchnls = 1
        0dbfs = 2

        gibasekey = 31 ; G
        ginterval = 12 ; default

        opcode midifreq, k, kk
        ; lets me express pitches in a more natural way, with separate
        ; octave number and note within octave, offset by a global base
        ; key.
        koct, knote xin
        kfreq = cpsmidinn(koct * 12 + knote * 12 / ginterval + gibasekey)
        xout kfreq
        endop

        instr 1
            kamp linseg 0, 0.02, 1, p3 - 0.04, 1, 0.02, 0
            kfreq midifreq p4, p5
            araw oscil kamp, kfreq
            out araw
        endin

        instr 10
            ; doesn’t produce any sound, but changes the number of
            ; divisions in the octave.
            ginterval = p4
            prints "%d divisions per octave\n", ginterval
        endin

    </CsInstruments>


Based on the above instrument definitions, the following Python function generates an ascending and descending scale using the specified number of divisions in the octave:

In [ ]:
def scale(divisions) :
    # outputs notes for an ascending and descending scale
    # over a single octave, with the specified number of
    # divisions of the octave.
    base_octave = 2
    result = \
        [
            "i10 0 0 %d" % divisions,
            "i1 0 1 %d 0" % base_octave,
        ]
    for i in range(1, divisions) :
        result.append("i1 + . %d %d" % (base_octave, i))
    #end for
    result.append("i1 + . %d 0" % (base_octave + 1))
    for i in range(divisions, 0, -1) :
        result.append("i1 + . %d %d" % (base_octave, i - 1))
    #end for
    return \
        result
#end scale

To start with, the following generates the conventional 12-semitone scale:

In [ ]:
%%csound
<CsoundSynthesizer>

%insval common_setup

    <CsScore>
        t0 180
%insval scale(12)
        e
    </CsScore>

</CsoundSynthesizer>


Now let’s try quarter-tones:

In [ ]:
%%csound
<CsoundSynthesizer>

%insval common_setup

    <CsScore>
        t0 180
%insval scale(24)
        e
    </CsScore>

</CsoundSynthesizer>


What do other divisions sound like? Feel free to try different values.